<summary>Table of Contents</summary>

- [1. No RevIN](#1-no-revin-instanse-normalization)
- [2. No channel-independence (Channel-Mixing)](#2-no-channel-independence-channel-mixing)
- [3. No Patching](#3-no-patching)
- [4. Time series decomposition](#4-ts-decomposition)

Ablation study on PatchTST components.

In [2]:
import os
import pandas as pd
import subprocess
import shutil
import time
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

# 1. No RevIN (Instanse Normalization)

In [3]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [7]:
cuda_device = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]
seq_lens = [512, 512, 336, 168, 168]

model = "PatchTST"
loss = "MSE"
itr=1

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_no_revin.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128

# List to store the results
patchtst_results = []

In [5]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"no_revin_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --revin 0 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===



KeyboardInterrupt: 

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['-RevIN'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_no_revin.csv'))
patchtst_df.round(4)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Model             -RevIN                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0218  0.1477  0.0952
        96        0.0363  0.1906  0.1306
        168       0.0390  0.1975  0.1373
ES      24        0.0184  0.1355  0.0841
        96        0.0310  0.1761  0.1143
        168       0.0329  0.1814  0.1212
FR      24        0.0110  0.1048  0.0640
        96        0.0194  0.1392  0.0873
        168       0.0214  0.1465  0.0929
GB      24        0.0266  0.1631  0.1095
        96        0.0406  0.2014  0.1431
        168       0.0482  0.2195  0.1556
IT      24        0.0109  0.1042  0.0658
        96        0.0179  0.1337  0.0860
        168       0.0189  0.1375  0.0897

# 2. No channel independence (Channel-Mixing)

It is a channel mixing model, and therefore it needs more dimension of embeddings to capture complex patterns between features. 

Therefore, it is not fair to keep same d_model and d_ff as in channel mixing. In this regard, we scale them based on number of input features.

In other words, for DE data with 5 columns, d_model = 128 x 5, and d_ff = 256 x 5.

For ES: d_model = 128 x 3 and d_ff = 256 x 3, etc. It is adjusted automatically in code.

Since it converges fast, we reduced max number of epochs and patience.

Complete results are in logs.

In [ ]:
# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_channel_mixing_MIX_FEATURES.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)
        
        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            if country == 'DE' and pred_len == 24:
                seq_len = 336
            else:
                seq_len = seq_lens[i]

            model_id = f"channel_mixing_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --channel_mixing 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===



Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='channel_mixing_DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=1, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, patience=10, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, use_amp=False, use_gpu=True, gpu

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['CM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_channel_mixing_MIX_FEATURES.csv'))
patchtst_df.round(4)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


Model                 CM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0255  0.1596  0.1060
        96        0.0381  0.1953  0.1384
        168       0.0400  0.2001  0.1428
ES      24        0.0114  0.1068  0.0692
        96        0.0207  0.1437  0.0965
        168       0.0229  0.1514  0.1032
FR      24        0.0110  0.1049  0.0611
        96        0.0191  0.1383  0.0849
        168       0.0200  0.1413  0.0889
GB      24        0.0286  0.1691  0.1175
        96        0.0428  0.2069  0.1471
        168       0.0450  0.2121  0.1520
IT      24        0.0107  0.1033  0.0620
        96        0.0188  0.1372  0.0860
        168       0.0198  0.1408  0.0902

In [ ]:
countries = ['DE', 'GB', 'ES']
num_cols = [5, 5, 3]

# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_channel_mixing_168.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)
        
        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 

            seq_len=168

            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --channel_mixing 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===



Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_168_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=1, seq_len=168, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, patience=10, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, use_amp=False, use_gpu=True, gpu=0, use_multi_g

In [ ]:
path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['CM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_channel_mixing_168.csv'))
patchtst_df.round(4)

Model                 CM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0234  0.1529  0.1006
        96        0.0443  0.2104  0.1429
        168       0.0436  0.2088  0.1491
ES      24        0.0116  0.1076  0.0693
        96        0.0208  0.1444  0.0970
        168       0.0229  0.1512  0.1022
GB      24        0.0266  0.1630  0.1094
        96        0.0464  0.2154  0.1514
        168       0.0494  0.2224  0.1574

In [ ]:
countries = ['ES']
num_cols = [3]

# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_channel_mixing_96.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)
        
        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 

            seq_len=96

            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --channel_mixing 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: ES ===


=== Starting experiments for pred_len: 24 ===



Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='ES_96_24_ES', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='ES_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=1, seq_len=96, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=7, c_out=3, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, patience=10, learning_rate=0.0001, des='Exp', lradj='type3', pct_start=0.3, use_amp=False, use_gpu=True, gpu=0, use_multi_gpu

In [ ]:
path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['CM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_channel_mixing_96.csv'))
patchtst_df.round(4)

Model                 CM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
ES      24        0.0116  0.1077  0.0687
        96        0.0213  0.1458  0.0983
        168       0.0236  0.1535  0.1039

# 3. No patching

It runs more than 24 hours on 48GB GPU (1 country around 5-6 hours). Therefore I run it with portions. You can find full results in logs.

In [8]:
# Initialize empty list
patchtst_results = []

patch_len = 1
stride = 1

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_no_patching.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 

            if country == 'DE' and pred_len == 24:
                seq_len=336
            else:
                seq_len = seq_lens[i]

            if seq_len == 512:
                batch_size = 64
            else:
                batch_size = 128
                
            model_id = f"no_patching_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"

            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len {patch_len} \
              --stride {stride} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='no_patching_DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=1, stride=1, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=20, batch_size=128, patience=5, learni

KeyboardInterrupt: 

In [9]:
patchtst_results

[{'Country': 'DE',
  'Pred_len': 24,
  'Iteration': 1,
  'MSE': 0.021901745349168777,
  'RMSE': 0.14799238741397858,
  'MAE': 0.09500051289796829},
 {'Country': 'DE',
  'Pred_len': 96,
  'Iteration': 1,
  'MSE': 0.03542259708046913,
  'RMSE': 0.1882089227437973,
  'MAE': 0.13172055780887604},
 {'Country': 'DE',
  'Pred_len': 168,
  'Iteration': 1,
  'MSE': 0.037591367959976196,
  'RMSE': 0.193884938955307,
  'MAE': 0.13648755848407745},
 {'Country': 'GB',
  'Pred_len': 24,
  'Iteration': 1,
  'MSE': 0.02572702430188656,
  'RMSE': 0.16039645671844482,
  'MAE': 0.1057143285870552},
 {'Country': 'GB',
  'Pred_len': 96,
  'Iteration': 1,
  'MSE': 0.0402989462018013,
  'RMSE': 0.20074597001075745,
  'MAE': 0.14150388538837433},
 {'Country': 'GB',
  'Pred_len': 168,
  'Iteration': 1,
  'MSE': 0.04305318742990494,
  'RMSE': 0.2074926197528839,
  'MAE': 0.14702880382537842}]

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['- P'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_no_patching.csv'))
patchtst_df.round(4)

Model                - P                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0217  0.1472  0.0909
        96        0.0365  0.1910  0.1267
        168       0.0384  0.1959  0.1325
ES      24        0.0107  0.1034  0.0632
        96        0.0193  0.1390  0.0894
        168       0.0217  0.1472  0.0960
FR      24        0.0108  0.1037  0.0574
        96        0.0201  0.1418  0.0822
        168       0.0219  0.1478  0.0877
GB      24        0.0263  0.1621  0.1036
        96        0.0425  0.2061  0.1391
        168       0.0442  0.2102  0.1445
IT      24        0.0104  0.1021  0.0587
        96        0.0187  0.1367  0.0815
        168       0.0199  0.1410  0.0857

# 4. TS Decomposition

In [ ]:
# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_decomposition.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            if country == "DE" and pred_len == 24:
                seq_len = 336
            else:
                seq_len = seq_lens[i]
                
            model_id = f"ts_decomp_{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --decomposition 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            #shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: FR ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='ts_decomp_FR_168_24_FR', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='FR_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=168, label_len=48, pred_len=24, inverse=False, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=1, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=7, c_out=3, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=100, batch_size=128, patience=10, learn

In [ ]:
#shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['Decomposition'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_decomposition.csv'))
patchtst_df.round(4)

Model            Decomposition                
Metrics                    MSE    RMSE     MAE
Country Pred_len                              
DE      24              0.0212  0.1455  0.0921
        96              0.0350  0.1870  0.1285
        168             0.0376  0.1940  0.1374
ES      24              0.0100  0.0998  0.0622
        96              0.0185  0.1362  0.0902
        168             0.0209  0.1446  0.0967
FR      24              0.0099  0.0997  0.0569
        96              0.0192  0.1385  0.0838
        168             0.0208  0.1442  0.0896
GB      24              0.0257  0.1604  0.1056
        96              0.0442  0.2102  0.1458
        168             0.0433  0.2082  0.1481
IT      24              0.0099  0.0996  0.0583
        96              0.0178  0.1334  0.0818
        168             0.0190  0.1380  0.0874